<a href="https://colab.research.google.com/github/ivonnics/Machine-Learning/blob/master/Diferencias_en_Excel_(Hecho_con_Data_Real).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Updated: Using Pandas to Create and Excel Diff

Taken from: https://pbpython.com/excel-diff-pandas-update.html

Data from: https://github.com/chris1610/pbpython/tree/master/data

and

https://github.com/chris1610/pbpython/blob/master/notebooks/Excel_Diff_Update.ipynb

The [original article](http://pbpython.com/excel-diff-pandas.html) contains some Updating the Excel diff article to work with more recent versions of pandas that no longer use panel.



In [0]:
import pandas as pd

In [0]:
# Define the diff function to show the changes in each field
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)

## Datasets TestCompA.xlsx & TestCompB.xlsx en: https://github.com/ivonnics/Data-sets

In [0]:
# Read in the two files but call the data old and new and create columns to track
#old = pd.read_excel('sample-address-1.xlsx', 'Sheet1', na_values=['NA'])
old = pd.read_excel('TestCompA.xlsx', 'Sheet2', na_values=['NA'])
#new = pd.read_excel('sample-address-2.xlsx', 'Sheet1', na_values=['NA'])
new = pd.read_excel('TestCompB.xlsx', 'Sheet2', na_values=['NA'])
old.drop(old.index[0],inplace=True)
new.drop(new.index[0],inplace=True)
old['version'] = "old"
new['version'] = "new"

In [0]:
old.head()

In [0]:
new.head()

In [0]:
old['index1'] = old.index
new['index1'] = new.index
old.rename(columns={'Column1': 'Date', 'Column2': 'Desc', 'Column3':'Debit','Column4':'Credit','Column5':'Category'	},inplace=True)
new.rename(columns={'Column1': 'Date', 'Column2': 'Desc', 'Column3':'Debit','Column4':'Credit','Column5':'Category'	},inplace=True)

In [0]:
old.head()

In [0]:
new.head()

In [0]:
# We use the account numbers as the keys to check what is added, dropped and potentially changed
# Using sets makes the deduping easy and we can use set operations to figure out groupings
old_accts_all = set(old['index1'])
new_accts_all = set(new['index1'])

dropped_accts = old_accts_all - new_accts_all
added_accts = new_accts_all - old_accts_all

In [0]:
#Join all the data together and ignore indexes so it all gets concatenated
all_data = pd.concat([old,new],ignore_index=True)

In [0]:
all_data.head()

In [0]:
# Let's see what changes in the main columns we care about
# Change drop_duplicates syntax: keep=last
changes = all_data.drop_duplicates(subset=["Date", 
                                           "Desc", "Debit", 
                                           "Credit","Category"], keep='last')

In [0]:
changes.head()

In [0]:
#Get all the duplicate rows
dupe_accts = changes[changes['index1'].duplicated() == True]['index1'].tolist()
dupes = changes[changes["index1"].isin(dupe_accts)]

In [0]:
dupes

In [0]:
# Pull out the old and new data into separate dataframes
change_new = dupes[(dupes["version"] == "new")]
change_old = dupes[(dupes["version"] == "old")]

In [0]:
# Drop the temp columns - we don't need them now
change_new = change_new.drop(['version'], axis=1)
change_old = change_old.drop(['version'], axis=1)

In [0]:
# Index on the account numbers
change_new.set_index('index1', inplace=True)
change_old.set_index('index1', inplace=True)

In [0]:
df_all_changes = pd.concat([change_old, change_new],
                           axis='columns',
                           keys=['old', 'new'],
                           join='outer')

In [0]:
df_all_changes

In [0]:
df_all_changes = df_all_changes.swaplevel(axis='columns')[change_new.columns[0:]]

In [0]:
df_all_changes

In [0]:
df_changed = df_all_changes.groupby(level=0, axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))
df_changed = df_changed.reset_index()

In [0]:
df_changed

In [0]:
# Diff'ing is done, we need to get a list of removed and added items

In [0]:
df_removed = changes[changes["index1"].isin(dropped_accts)]
df_removed

In [0]:
df_added = changes[changes["index1"].isin(added_accts)]
df_added

In [0]:
#Save the changes to excel but only include the columns we care about
output_columns = ['Date','Desc',	'Debit',	'Credit',	'Category']
writer = pd.ExcelWriter("my-diff2.xlsx")
df_changed.to_excel(writer,"changed", index=False, columns=output_columns)
df_removed.to_excel(writer,"removed",index=False, columns=output_columns)
df_added.to_excel(writer,"added",index=False, columns=output_columns)
writer.save()